# Analyse inférentielle d'échantillonnage 

 Nettoyage des colonnes avec les valeurs nulles 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats


In [0]:
df = pd.read_csv("/Volumes/workspace/trips/sample/yellowtaxisample1pct_hybrid_stratified.csv")
df.head()


# EDA 

In [0]:
# savoir la taille des donnees 
taille = df.shape
taille 

In [0]:
nom_colonnes = df.columns

print(f'Les noms des colonnes sont :\n{nom_colonnes.tolist()}')


In [0]:
df.info()

In [0]:
# valeurs manquantes par colonne
valeurs_nulles = df.isnull().sum()
taux_valeurs_nulles = round((df.isnull().sum() / len(df)) * 100, 2)

df_nulles = pd.DataFrame({
    'Colonne': valeurs_nulles.index,
    'Nb_Manquants': valeurs_nulles.values,
    'Pourcentage': taux_valeurs_nulles.values
}).sort_values('Nb_Manquants', ascending=False)

print(df_nulles[df_nulles['Nb_Manquants'] > 0])

In [0]:
df.describe()

In [0]:
df.duplicated()

In [0]:
df.isnull().sum()

# Analyse des incoherences

In [0]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])


In [0]:
 #duree de la course : (en minutes)
df["trip_duration_min"] = (
    df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
).dt.total_seconds() / 60

# duree de course anormale
out_duration_negatif = df[(df["trip_duration_min"] < 0)]
out_duration_null = df[(df["trip_duration_min"] == 0)]


len(out_duration_negatif), len(out_duration_null)

In [0]:
# les dates sont cohérentes?
out_dates_drop_avant_pickup = df[
    (df["tpep_dropoff_datetime"] < df["tpep_pickup_datetime"])
]

out_dates_drop_pickup_egaux = df[
    (df["tpep_dropoff_datetime"] == df["tpep_pickup_datetime"])
]

len(out_dates_drop_avant_pickup), len(out_dates_drop_pickup_egaux)

In [0]:
# distance anormale
out_trip_distance_negatif = df[
    df["trip_distance"] < 0
]

out_trip_distance_null = df[
    df["trip_distance"] == 0
]

len(out_trip_distance_negatif), len(out_trip_distance_null)

In [0]:
# nombres de passagers
value_counts = df['passenger_count'].value_counts(dropna=False)
value_percent = round(df['passenger_count'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
print(len(df[df['passenger_count'] >= 7]))

In [0]:
# type de tarif
value_counts = df['RatecodeID'].value_counts(dropna=False)
value_percent = round(df['RatecodeID'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
print(len(df[df['RatecodeID'] == 99]))

In [0]:
# fare amount anormal
out_fare_amount_negatif = df[(df["fare_amount"] < 0)]
out_fare_amount_null = df[(df["fare_amount"] == 0)]


print(df['fare_amount'].quantile(0.99),'/', len(df[df['fare_amount'] > df['fare_amount'].quantile(0.99)]))
print('*'*70)
len(out_fare_amount_negatif), len(out_fare_amount_null)

In [0]:
# mta_tax anormal 
value_counts = df['mta_tax'].value_counts(dropna=False)
value_percent = round(df['mta_tax'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
out_mta_tax_negatif = df[(df["mta_tax"] < 0)]
print(df['mta_tax'].quantile(0.99),'/', len(df[df['mta_tax'] > df['mta_tax'].quantile(0.99)]))
print('*'*70)
len(out_mta_tax_negatif)

In [0]:
# tip_amount anormal
out_tip_amount_negatif = df[(df["tip_amount"] < 0)]

print(df['tip_amount'].quantile(0.99), len(df[df['tip_amount'] > df['tip_amount'].quantile(0.99)]))
print('*'*70)
len(out_tip_amount_negatif)

In [0]:
# tolls amount anormal
out_tolls_amount_negatif = df[(df["tolls_amount"] < 0)]

print(df['tolls_amount'].quantile(0.99),'/', len(df[df['tolls_amount'] > df['tolls_amount'].quantile(0.99)]))
print('*'*70)
len(out_tolls_amount_negatif)

In [0]:
# improvement surcharge anormal
value_counts = df['improvement_surcharge'].value_counts(dropna=False)
value_percent = round(df['improvement_surcharge'].value_counts(normalize=True, dropna=False) * 100, 2)

result = pd.DataFrame({
    "Valeur": value_counts.index,
    "Nombre": value_counts.values,
    "Pourcentage (%)": value_percent.values
})

print(result)
print('*'*70)
out_improvement_surcharge_negatif = df[(df["improvement_surcharge"] < 0)]
print(df['improvement_surcharge'].quantile(0.99),'/', len(df[df['improvement_surcharge'] >  df['improvement_surcharge'].quantile(0.99)]))
print('*'*70)
len(out_improvement_surcharge_negatif) 

In [0]:
# total_amount anormal
out_total_amount_negatif = df[(df["total_amount"] < 0)]
out_total_amount_null = df[(df["total_amount"] == 0)]

total_amount_sum = df[df['total_amount'] != df['fare_amount'] + df['extra'] + df['mta_tax'] + df['tip_amount'] + df['tolls_amount'] + df['improvement_surcharge'] + df['congestion_surcharge']]

print(df['tolls_amount'].quantile(0.99),'/', len(df[df['total_amount'] > df['total_amount'].quantile(0.99)]))
print('*'*70)
len(out_tolls_amount_negatif), len(out_total_amount_null), len(total_amount_sum)



In [0]:
# incoherent combinees 
out_combined = {
    "distance_zero_fare_pos": len(df[(df["trip_distance"] == 0) & (df["fare_amount"] > 0)]),
    "distance_zero_duration_pos": len(df[(df["trip_distance"] == 0) & (df["trip_duration_min"] > 0)]),
    "tip_superieur_fare": len(df[df["tip_amount"] > df["fare_amount"]])
}

out_combined


# visualization des donneés

In [0]:
cols = [
    "RatecodeID",
    "payment_type",
    "store_and_fwd_flag",
    "passenger_count"
]

for col in cols:
    print(f"\n Distribution de la colonne : {col}")

    # Comptage
    value_counts = df[col].value_counts(dropna=False)
    value_percent = round(df[col].value_counts(normalize=True, dropna=False) * 100, 2)

    result = pd.DataFrame({
        "Valeur": value_counts.index,
        "Nombre": value_counts.values,
        "Pourcentage (%)": value_percent.values
    })

    print(result)

    # Visualisation
    plt.figure(figsize=(10, 4))
    value_counts.plot(kind="barh")
    plt.title(f"Distribution de {col}")
    plt.xlabel("Nombre d'occurrences")
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

# Remplacer les valeures nulles 


In [0]:
df.loc[
    (df["airport_fee"].isna()) & (~df["PULocationID"].isin([132, 138])),
    "airport_fee"
] = 0


In [0]:
df["passenger_count"] = df["passenger_count"].fillna(
    df.groupby("PULocationID")["passenger_count"].transform("median")
)


In [0]:
df["RatecodeID"] = df["RatecodeID"].fillna(
    df.groupby("PULocationID")["RatecodeID"]
      .transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
)


In [0]:
df["store_and_fwd_flag"] = df["store_and_fwd_flag"].fillna("N")


In [0]:
df.isnull().sum()

In [0]:
money_cols = [
    "congestion_surcharge",
    "airport_fee",
    "Airport_fee"
]

df[money_cols] = df[money_cols].fillna(0)


In [0]:
df.isnull().sum()

In [0]:
df[df["PULocationID"]==59]


In [0]:
money_cols = [
    "passenger_count"  ,
    "RatecodeID ",
    
]

df["passenger_count"] = 1
df["RatecodeID"] = 1


In [0]:
df.isnull().sum()

In [0]:
from pyspark.sql.functions import (
    col, count, mean, stddev, sum as spark_sum, min as spark_min, max as spark_max,
    hour, dayofweek, dayofmonth, month, year, weekofyear,
    unix_timestamp, percentile_approx, when, lit
)
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df= spark.read.csv("/Volumes/workspace/trips/sample/yellowtaxisample1pct_hybrid_stratified.csv",
    header=True,
    inferSchema=True
)
nb_sample = df.count()
print(f" Échantillon chargé : {nb_sample:,} courses")

In [0]:


# COMMAND ----------
import numpy as np
import pandas as pd
from scipy.stats import t, norm
import scipy.stats as stats
from pyspark.sql.functions import col, count, mean, stddev
from pyspark.sql.functions import unix_timestamp



# Conversion en pandas pour calculs statistiques
fare_sample_pd = df.select("fare_amount").toPandas()['fare_amount']

# Statistiques de base
n = len(fare_sample_pd)
mean_fare = fare_sample_pd.mean()
std_fare = fare_sample_pd.std()
se_fare = std_fare / np.sqrt(n)

# Intervalle de confiance 95% (distribution t de Student)
confidence_level = 0.95
alpha = 1 - confidence_level
df_freedom = n - 1
t_critical = t.ppf(1 - alpha/2, df_freedom)

margin_error = t_critical * se_fare
ic_inf = mean_fare - margin_error
ic_sup = mean_fare + margin_error

print("=== PRIX MOYEN (FARE_AMOUNT) - INFÉRENCE ===")
print(f"Taille échantillon: {n:,}")
print(f"Prix moyen estimé: ${mean_fare:.2f}")
print(f"Écart-type: ${std_fare:.2f}")
print(f"Erreur standard: ${se_fare:.4f}")
print(f"t-critique (95%): {t_critical:.3f}")
print(f"Marge d'erreur: ${margin_error:.2f}")
print(f"\n IC 95%: [${ic_inf:.2f}, ${ic_sup:.2f}]")

# Stockage pour comparaison
results_inferential = {
    'mean_fare': mean_fare,
    'ic_fare_inf': ic_inf,
    'ic_fare_sup': ic_sup
}



# Distance moyenne
distance_sample_pd = df.select("trip_distance").toPandas()['trip_distance']

mean_distance = distance_sample_pd.mean()
std_distance = distance_sample_pd.std()
se_distance = std_distance / np.sqrt(n)
margin_error_dist = t_critical * se_distance

ic_dist_inf = mean_distance - margin_error_dist
ic_dist_sup = mean_distance + margin_error_dist

print("=== DISTANCE MOYENNE (TRIP_DISTANCE) - INFÉRENCE ===")
print(f"Distance moyenne estimée: {mean_distance:.2f} miles")
print(f"Écart-type: {std_distance:.2f}")
print(f" IC 95%: [{ic_dist_inf:.2f}, {ic_dist_sup:.2f}] miles")

results_inferential['mean_distance'] = mean_distance
results_inferential['ic_dist_inf'] = ic_dist_inf
results_inferential['ic_dist_sup'] = ic_dist_sup



# Calcul de la durée en minutes
df_sample_duration = df.withColumn(
    "duration_minutes",
    (col("tpep_dropoff_datetime").cast("long")
     - col("tpep_pickup_datetime").cast("long")) / 60
)

duration_sample_pd = df_sample_duration.select("duration_minutes").toPandas()['duration_minutes']

mean_duration = duration_sample_pd.mean()
std_duration = duration_sample_pd.std()
se_duration = std_duration / np.sqrt(n)
margin_error_dur = t_critical * se_duration

ic_dur_inf = mean_duration - margin_error_dur
ic_dur_sup = mean_duration + margin_error_dur

print("=== DURÉE MOYENNE - INFÉRENCE ===")
print(f"Durée moyenne estimée: {mean_duration:.2f} minutes")
print(f"Écart-type: {std_duration:.2f}")
print(f" IC 95%: [{ic_dur_inf:.2f}, {ic_dur_sup:.2f}] minutes")

results_inferential['mean_duration'] = mean_duration
results_inferential['ic_dur_inf'] = ic_dur_inf
results_inferential['ic_dur_sup'] = ic_dur_sup



# Proportion dans l'échantillon
nb_with_tip = df.filter(col("tip_amount") > 0).count()
p_sample = nb_with_tip / n

# IC pour proportion (approximation normale)
z_critical = norm.ppf(1 - alpha/2)  # 1.96 pour 95%
se_prop = np.sqrt(p_sample * (1 - p_sample) / n)
margin_error_prop = z_critical * se_prop

ic_prop_inf = max(0, p_sample - margin_error_prop)
ic_prop_sup = min(1, p_sample + margin_error_prop)

print("=== PROPORTION AVEC TIP > 0 - INFÉRENCE ===")
print(f"Nombre avec tip: {nb_with_tip:,}")
print(f"Proportion estimée: {p_sample:.2%}")
print(f"z-critique (95%): {z_critical:.3f}")
print(f" IC 95%: [{ic_prop_inf:.2%}, {ic_prop_sup:.2%}]")

results_inferential['prop_tip'] = p_sample
results_inferential['ic_prop_inf'] = ic_prop_inf
results_inferential['ic_prop_sup'] = ic_prop_sup



# Calcul du ratio tip/fare pour échantillon
df_sample_ratio = df.filter(
    (col("fare_amount") > 0) & (col("tip_amount") >= 0)
).withColumn(
    "tip_ratio", col("tip_amount") / col("fare_amount")
)

# Groupement par type de paiement
ratio_by_payment = df_sample_ratio.groupBy("payment_type").agg(
    count("*").alias("nb_courses"),
    mean("tip_ratio").alias("ratio_tip_fare_moyen"),
    stddev("tip_ratio").alias("std_ratio")
).orderBy(col("nb_courses").desc())

print("=== RATIO TIP/FARE PAR TYPE DE PAIEMENT - INFÉRENCE ===")
display(ratio_by_payment)

# Pour chaque type de paiement, calculer IC
ratio_pd = df_sample_ratio.select("payment_type", "tip_ratio").toPandas()
for payment_type in ratio_pd['payment_type'].unique():
    data = ratio_pd[ratio_pd['payment_type'] == payment_type]['tip_ratio']
    if len(data) > 30:  # Nombre suffisant
        mean_r = data.mean()
        std_r = data.std()
        n_r = len(data)
        se_r = std_r / np.sqrt(n_r)
        me_r = t.ppf(0.975, n_r-1) * se_r
        print(f"Payment {payment_type}: {mean_r:.2%} ± {me_r:.2%}")



from pyspark.sql.functions import col
import scipy.stats as stats

# Filtrage des prix valides
card_fares = df.filter(
    (col("payment_type") == 1) & (col("fare_amount") > 0)
).select("fare_amount").toPandas()['fare_amount']

cash_fares = df.filter(
    (col("payment_type") == 2) & (col("fare_amount") > 0)
).select("fare_amount").toPandas()['fare_amount']

# Test t de Student (échantillons indépendants)
if len(card_fares) > 30 and len(cash_fares) > 30:
    t_stat, p_value = stats.ttest_ind(card_fares, cash_fares, equal_var=False)

    print("=== TEST D'HYPOTHÈSE : CARTE vs CASH ===")
    print(f"Carte - Prix moyen: ${card_fares.mean():.2f} (n={len(card_fares)})")
    print(f"Cash  - Prix moyen: ${cash_fares.mean():.2f} (n={len(cash_fares)})")
    print(f"\nt-statistique: {t_stat:.3f}")
    print(f"p-value: {p_value:.6f}")

    if p_value < 0.05:
        print("\n Différence SIGNIFICATIVE (p < 0.05)")
        print("Le mode de paiement influence le prix moyen de la course")
    else:
        print("\n Différence NON significative (p ≥ 0.05)")
        print(" Aucune preuve d’un effet du mode de paiement")
else:
    print(" Taille d’échantillon insuffisante pour le test")


# COMMAND ----------

In [0]:
clean_df=df.copy()

### DISTRIBUTION DES COURSES

In [0]:
# Extraction des composantes temporelles
clean_df["pickup_hour"] = clean_df["tpep_pickup_datetime"].dt.hour
clean_df["pickup_dayofweek"] = clean_df["tpep_pickup_datetime"].dt.day_name()
clean_df["pickup_week"] = clean_df["tpep_pickup_datetime"].dt.isocalendar().week

In [0]:
# Distribution des courses par heure
hourly_counts = clean_df["pickup_hour"].value_counts().sort_index()

hourly_prop = hourly_counts / hourly_counts.sum()

In [0]:
# Intervalle de confiance (proportion horaire)
hour = 18
k = hourly_counts.loc[hour]
n = len(clean_df)

p_hat = k / n
z = stats.norm.ppf(0.975)
se = np.sqrt((p_hat * (1 - p_hat)) / n)

ci_low = p_hat - z * se
ci_high = p_hat + z * se

print(f"Heure {hour}h :")
print(f"Proportion : {p_hat:.4f}")
print(f"IC 95% : [{ci_low:.4f}, {ci_high:.4f}]")

In [0]:
# Distribution par jour de la semaine
day_counts = clean_df["pickup_dayofweek"].value_counts().sort_index()

day_labels = {
    0: "Lundi", 1: "Mardi", 2: "Mercredi",
    3: "Jeudi", 4: "Vendredi", 5: "Samedi", 6: "Dimanche"
}

day_counts.rename(index=day_labels)

In [0]:
# Distribution hebdomadaire (saisonnalité légère)
weekly_counts = clean_df["pickup_week"].value_counts().sort_index()

plt.figure(figsize=(10, 5))
hourly_counts.plot(kind="bar")
plt.title("Distribution des courses par heure")
plt.xlabel("Heure de pickup")
plt.ylabel("Nombre de courses")
plt.show()

###  COMPARAISON DES FARES SELON LES ZONES GÉOGRAPHIQUES

In [0]:
# Prix moyen par borough (pickup)
fare_by_borough = (
    clean_df
    .groupby("PULocationID")["fare_amount"]
    .agg(["count", "mean", "std"])
    .reset_index()
)

In [0]:
# Intervalle de confiance à 95 % par borough
def confidence_interval(mean, std, n, alpha=0.05):
    if n <= 1:
        return (np.nan, np.nan)
    t = stats.t.ppf(1 - alpha/2, df=n-1)
    margin = t * std / np.sqrt(n)
    return mean - margin, mean + margin


fare_by_borough["ci_low"], fare_by_borough["ci_high"] = zip(*fare_by_borough.apply(
    lambda row: confidence_interval(row["mean"], row["std"], row["count"]),
    axis=1
))
fare_by_borough.sort_values("mean", ascending=False)